DS Lab Assignment 8

Kadali Sai Vivek

197139

Implement neural network from scratch using python for the following datasets and predict the values for the following datasets

In [16]:
#Importing Pacakges
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.metrics import accuracy_score            


In [17]:
#Implementation of ANN from Scratch 

# ReLu Activation Function 
def relu(z):
    a = np.maximum(0,z)
    return a

def initialize_params(layer_sizes):
    params = {}
    for i in range(1, len(layer_sizes)):
        params['W' + str(i)] = np.random.randn(layer_sizes[i], layer_sizes[i-1])*0.01
        params['B' + str(i)] = np.random.randn(layer_sizes[i],1)*0.01
    return params

def forward_propagation(X_train, params):
    layers = len(params)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], X_train) + params['B' + str(i)]
            values['A' + str(i)] = relu(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], values['A' + str(i-1)]) + params['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = relu(values['Z' + str(i)])
    return values

def compute_cost(values, Y_train):
    layers = len(values)//2
    Y_pred = values['A' + str(layers)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

def backward_propagation(params, values, X_train, Y_train):
    layers = len(params)//2
    m = len(Y_train)
    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            dA = 1/m * (values['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(params['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(values['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,values['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads

def update_params(params, grads, learning_rate):
    layers = len(params)//2
    params_updated = {}
    for i in range(1,layers+1):
        params_updated['W' + str(i)] = params['W' + str(i)] - learning_rate * grads['W' + str(i)]
        params_updated['B' + str(i)] = params['B' + str(i)] - learning_rate * grads['B' + str(i)]
    return params_updated

def model(X_train, Y_train, layer_sizes, num_iters, learning_rate):
    params = initialize_params(layer_sizes)
    for i in range(num_iters):
        values = forward_propagation(X_train.T, params)
        cost = compute_cost(values, Y_train.T)
        grads = backward_propagation(params, values,X_train.T, Y_train.T)
        params = update_params(params, grads, learning_rate)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return params

def compute_accuracy(X_train, X_test, Y_train, Y_test, params):
    values_train = forward_propagation(X_train.T, params)
    values_test = forward_propagation(X_test.T, params)
    train_acc = np.sqrt(mean_squared_error(Y_train, values_train['A' + str(len(layer_sizes)-1)].T))
    test_acc = np.sqrt(mean_squared_error(Y_test, values_test['A' + str(len(layer_sizes)-1)].T))
    return train_acc, test_acc

def predict(X, params):
    values = forward_propagation(X.T, params)
    predictions = values['A' + str(len(values)//2)].T
    return predictions



Q1) Boston House price Dataset

In [6]:
from google.colab import files
uploaded=files.upload()

Saving boston.csv to boston.csv


In [7]:
dataset= pd.read_csv('boston.csv')           
dataset

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [8]:
# Number of Columns
n=dataset.shape[1]    
# Number of Rows           
m=dataset.shape[0]  

layer_sizes=[13,5,5,1]

x_train=np.array(dataset.iloc[:int(0.8*m),:n-1])      
y_train=np.array(dataset.iloc[:int(0.8*m),-1])        
x_test=np.array(dataset.iloc[int(0.8*m):,:n-1])       
y_test=np.array(dataset.iloc[int(0.8*m):,-1])         

#Creating the Model
params = model(x_train, y_train, layer_sizes, num_iters=1000, learning_rate=0.03)
#Predicting the Values 1
y_predict = predict(x_test,params)
print(y_predict)

Cost at iteration 1 = 335.19541497393453

Cost at iteration 2 = 335.15114700349704

Cost at iteration 3 = 335.1068353483903

Cost at iteration 4 = 335.06248056245556

Cost at iteration 5 = 335.01807876394946

Cost at iteration 6 = 334.97362436456956

Cost at iteration 7 = 334.9291179924516

Cost at iteration 8 = 334.8845444585521

Cost at iteration 9 = 334.839901969437

Cost at iteration 10 = 334.79520155746195

Cost at iteration 11 = 334.7504371609849

Cost at iteration 12 = 334.7056018382812

Cost at iteration 13 = 334.6606892389871

Cost at iteration 14 = 334.6156888503963

Cost at iteration 15 = 334.5705910458843

Cost at iteration 16 = 334.52538577265494

Cost at iteration 17 = 334.4800634476489

Cost at iteration 18 = 334.4346129834435

Cost at iteration 19 = 334.389021345713

Cost at iteration 20 = 334.343273532257

Cost at iteration 21 = 334.29735401433186

Cost at iteration 22 = 334.2512467976269

Cost at iteration 23 = 334.20493199154237

Cost at iteration 24 = 334.1583877465

Q2) The seeds dataset

In [9]:
from google.colab import files
uploaded=files.upload()

Saving seeds_dataset.csv to seeds_dataset.csv


In [12]:
dataset= pd.read_csv('seeds_dataset.csv')         
dataset 

,15.26,14.84,0.871,5.763,3.312,2.221,5.22,1
0,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
1,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
2,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
3,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
4,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1
...,...,...,...,...,...,...,...,...
204,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
205,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
206,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
207,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [20]:
# Number of Columns
n=dataset.shape[1]    
# Number of Rows           
m=dataset.shape[0]  

layer_sizes=[7,5,5,1]


x_train=np.array(dataset.iloc[:int(0.8*m),:n-1])      
y_train=np.array(dataset.iloc[:int(0.8*m),-1])        
x_test=np.array(dataset.iloc[int(0.8*m):,:n-1])       
y_test=np.array(dataset.iloc[int(0.8*m):,-1])         

#Creating the Model
params = model(x_train, y_train, layer_sizes, num_iters=1000, learning_rate=0.03)
#Predicting the Values 1
y_predict = predict(x_test,params)
print("Y prediction :", y_predict)
print("Y test data :",y_test)


Cost at iteration 1 = 1.8133041929882383

Cost at iteration 2 = 1.8127458595103867

Cost at iteration 3 = 1.812187726471902

Cost at iteration 4 = 1.8116297938006851

Cost at iteration 5 = 1.8110720614246636

Cost at iteration 6 = 1.8105145292717897

Cost at iteration 7 = 1.8099571972700423

Cost at iteration 8 = 1.8094000653474263

Cost at iteration 9 = 1.8088431334319726

Cost at iteration 10 = 1.808286401451738

Cost at iteration 11 = 1.8077298693348043

Cost at iteration 12 = 1.8071735370092807

Cost at iteration 13 = 1.8066174044033008

Cost at iteration 14 = 1.806061471445026

Cost at iteration 15 = 1.8055057380626411

Cost at iteration 16 = 1.8049502041843581

Cost at iteration 17 = 1.8043948697384153

Cost at iteration 18 = 1.8038397346530763

Cost at iteration 19 = 1.8032847988566305

Cost at iteration 20 = 1.8027300622773927

Cost at iteration 21 = 1.8021755248437041

Cost at iteration 22 = 1.801621186483931

Cost at iteration 23 = 1.8010670471264674

Cost at iteration 24 = 1